In [1]:
import pandas as pd
import numpy as np
import ssl

ssl._create_default_https_context = ssl._create_unverified_context # Para que la URL del archivo cvs funcione en mac
ruta = "https://raw.githubusercontent.com/lbtriana/Proyecto_Bicicletas/main/SeoulBikeData_utf8.csv" #ruta desde url
df = pd.read_csv(ruta)
df['fecha'] = pd.to_datetime(df['Date'], format='%d/%m/%Y')
df['day'] = df['fecha'].dt.day
df['month'] = df['fecha'].dt.month
df['year'] = df['fecha'].dt.year
df['day_of_week'] = df['fecha'].dt.dayofweek
df['functioning_day'] = df['Functioning Day'].replace({'Yes': 1, 'No': 0})
df['is_holiday'] = df['Holiday'].replace({'Holiday': 1, 'No Holiday': 0})
pd.set_option('future.no_silent_downcasting', True) # Esta linea es para evitar un warning de perder demasiada info al hacer downcasting
df['Hour_PM'] = np.where(df['Hour'] >= 12, 1, 0) #1 si es hora en la tarde 
df['Hour_lab'] = np.where((df['Hour'] >= 8) & (df['Hour'] <= 17), 1, 0) #1 si es horario laboral

df.info()

var_continuas = [
    'Hour', 
    'Temperature(C)', 
    'Humidity(%)', 
    'Wind speed (m/s)', 
    'Visibility (10m)', 
    'Dew point temperature(C)', 
    'Solar Radiation (MJ/m2)', 
    'Rainfall(mm)', 
    'Snowfall (cm)', 
    'day', 
    'month', 
    'year', 
    'day_of_week',
    'fecha',
    'Rented Bike Count'

 ]

var_categoricas = [
    'Seasons', 
    'Hour_PM', 
    'Hour_lab',
    'functioning_day', 
    'is_holiday' 
]

continuas=df[var_continuas]
categoricas=df[var_categoricas]

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8760 entries, 0 to 8759
Data columns (total 23 columns):
 #   Column                    Non-Null Count  Dtype         
---  ------                    --------------  -----         
 0   Date                      8760 non-null   object        
 1   Rented Bike Count         8760 non-null   int64         
 2   Hour                      8760 non-null   int64         
 3   Temperature(C)            8760 non-null   float64       
 4   Humidity(%)               8760 non-null   int64         
 5   Wind speed (m/s)          8760 non-null   float64       
 6   Visibility (10m)          8760 non-null   int64         
 7   Dew point temperature(C)  8760 non-null   float64       
 8   Solar Radiation (MJ/m2)   8760 non-null   float64       
 9   Rainfall(mm)              8760 non-null   float64       
 10  Snowfall (cm)             8760 non-null   float64       
 11  Seasons                   8760 non-null   object        
 12  Holiday             

C:\Users\User\AppData\Local\Temp\ipykernel_24032\889776224.py:13: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df['functioning_day'] = df['Functioning Day'].replace({'Yes': 1, 'No': 0})
C:\Users\User\AppData\Local\Temp\ipykernel_24032\889776224.py:14: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df['is_holiday'] = df['Holiday'].replace({'Holiday': 1, 'No Holiday': 0})


In [2]:
df_cat = pd.get_dummies(categoricas, dtype="int64", drop_first=True)

df_cat.info()



<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8760 entries, 0 to 8759
Data columns (total 7 columns):
 #   Column           Non-Null Count  Dtype
---  ------           --------------  -----
 0   Hour_PM          8760 non-null   int64
 1   Hour_lab         8760 non-null   int64
 2   functioning_day  8760 non-null   int64
 3   is_holiday       8760 non-null   int64
 4   Seasons_Spring   8760 non-null   int64
 5   Seasons_Summer   8760 non-null   int64
 6   Seasons_Winter   8760 non-null   int64
dtypes: int64(7)
memory usage: 479.2 KB


In [3]:

df = pd.concat([continuas, df_cat], axis=1)

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8760 entries, 0 to 8759
Data columns (total 22 columns):
 #   Column                    Non-Null Count  Dtype         
---  ------                    --------------  -----         
 0   Hour                      8760 non-null   int64         
 1   Temperature(C)            8760 non-null   float64       
 2   Humidity(%)               8760 non-null   int64         
 3   Wind speed (m/s)          8760 non-null   float64       
 4   Visibility (10m)          8760 non-null   int64         
 5   Dew point temperature(C)  8760 non-null   float64       
 6   Solar Radiation (MJ/m2)   8760 non-null   float64       
 7   Rainfall(mm)              8760 non-null   float64       
 8   Snowfall (cm)             8760 non-null   float64       
 9   day                       8760 non-null   int32         
 10  month                     8760 non-null   int32         
 11  year                      8760 non-null   int32         
 12  day_of_week         

In [5]:
features = [
    'Hour', 
    'Temperature(C)', 
    'Humidity(%)', 
    'Wind speed (m/s)', 
    'Visibility (10m)',     
    'Dew point temperature(C)', 
    'Solar Radiation (MJ/m2)', 
    'Rainfall(mm)', 
    'Snowfall (cm)', 
    'Seasons_Winter',
    'Seasons_Spring',
    'Seasons_Summer',
    'day', 
    'month', 
    'year', 
    'day_of_week', 
    'is_holiday', 
    'Hour_PM', 
    'Hour_lab'
]

X = df[features]

Y = df['functioning_day']

from sklearn.model_selection import train_test_split 

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, random_state=1)

from sklearn.linear_model import LinearRegression

# crear el objeto del modelo
linreg = LinearRegression()

# ajustar los parámetros del modelo usando los datos de entrenamiento
linreg.fit(X_train, Y_train)



LinearRegression()

In [6]:
# imprimir coeficientes
print(linreg.intercept_)
print(linreg.coef_)


-15.258089455527582
[ 2.82270027e-04  1.57986244e-03  8.49182009e-04 -4.74948915e-03
 -2.08888539e-06 -4.12975253e-03 -3.12318572e-03 -1.12798498e-03
  1.31499445e-03  8.02615448e-02  9.19116007e-02  1.44678629e-01
  1.37569969e-03  9.30013173e-04  7.96064233e-03  1.18116790e-03
 -2.13358814e-02  9.40339210e-03  8.33117377e-03]


In [7]:
# coeficientes con nombre de las características
list(zip(features, linreg.coef_))


[('Hour', np.float64(0.0002822700269613784)),
 ('Temperature(C)', np.float64(0.0015798624395980771)),
 ('Humidity(%)', np.float64(0.0008491820085802025)),
 ('Wind speed (m/s)', np.float64(-0.004749489148140786)),
 ('Visibility (10m)', np.float64(-2.088885386823721e-06)),
 ('Dew point temperature(C)', np.float64(-0.00412975253276482)),
 ('Solar Radiation (MJ/m2)', np.float64(-0.003123185718944856)),
 ('Rainfall(mm)', np.float64(-0.0011279849762207885)),
 ('Snowfall (cm)', np.float64(0.0013149944477588368)),
 ('Seasons_Winter', np.float64(0.0802615447577704)),
 ('Seasons_Spring', np.float64(0.09191160067215066)),
 ('Seasons_Summer', np.float64(0.14467862863362227)),
 ('day', np.float64(0.0013756996937075532)),
 ('month', np.float64(0.0009300131725839993)),
 ('year', np.float64(0.00796064233330372)),
 ('day_of_week', np.float64(0.0011811678975378493)),
 ('is_holiday', np.float64(-0.021335881394364625)),
 ('Hour_PM', np.float64(0.009403392103975885)),
 ('Hour_lab', np.float64(0.00833117377

In [8]:
y_pred = linreg.predict(X_test)

In [9]:
from sklearn import metrics

In [175]:
import numpy as np

# mean absolute error
MAE = metrics.mean_absolute_error(Y_test, y_pred)

# mean squared error
MSE = metrics.mean_squared_error(Y_test, y_pred)

# root mean squared error
RMSE = np.sqrt(MSE)

In [176]:
print("MAE: ", MAE)
print("MSE: ", MSE)
print("RMSE: ", RMSE)

MAE:  339.20266164674194
MSE:  213637.68920109788
RMSE:  462.20957281421363


In [10]:
from sklearn.model_selection import cross_val_score

# usar MSE - error cuadrático medio
scores = cross_val_score(linreg, X, Y, cv=5, scoring='neg_mean_squared_error')
mse_scores = - scores
print(mse_scores)

[0.00076492 0.01338119 0.01439572 0.02523444 0.11532914]


In [178]:
# calcular RMSE
rmse_scores = np.sqrt(mse_scores)
print(rmse_scores)

[487.08963812 604.50592396 671.65679007 707.97337017 567.4537915 ]


In [179]:
# RMSE promedio a través de todos los grupos
print(rmse_scores.mean())

607.7359027635777


In [11]:
import statsmodels.api as sm

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, random_state=1)

# agregar constante explíticamente
X_train = sm.add_constant(X_train)

# regresión usando mínimos cuadrados ordinarios (ordinary least squares - OLS) 
model = sm.OLS(Y_train, X_train).fit()

# resumen de resultados
print(model.summary())




                            OLS Regression Results                            
Dep. Variable:        functioning_day   R-squared:                       0.079
Model:                            OLS   Adj. R-squared:                  0.076
Method:                 Least Squares   F-statistic:                     29.39
Date:                Tue, 10 Sep 2024   Prob (F-statistic):          1.21e-101
Time:                        10:47:47   Log-Likelihood:                 2245.1
No. Observations:                6570   AIC:                            -4450.
Df Residuals:                    6550   BIC:                            -4314.
Df Model:                          19                                         
Covariance Type:            nonrobust                                         
                               coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------------
const                   